In [59]:
# Analyzing frequency of certain aa for all telomeres to determine expected/common/uncommon
import pandas as pd

In [60]:
# reading in
telo5L = pd.read_excel('telomere-db.xlsx', sheet_name='5L').dropna()
telo12R = pd.read_excel('telomere-db.xlsx', sheet_name='12R').dropna()
telo17R = pd.read_excel('telomere-db.xlsx', sheet_name='17R').dropna()
telo19L = pd.read_excel('telomere-db.xlsx', sheet_name='19L').dropna()

In [61]:
telo17R[telo17R['Name']=='HG00621.pri.mat.f1_v2']

,Name,Mat/Pat,Population,Sex,Revcom?,Telomere,Frame1,Frame2,Frame3
23,HG00621.pri.mat.f1_v2,mat,Han Chinese South,male,no,17R,RVPVRVRVRG*GLGVRVKVNGQRQRSRSGSGSGVRVRVRGYGQRVG...,GFRFGFGFGVRG*GSGSRSTVSVSGQGQGQGQGSGLGSGVTVRG*G...,GSGSGSGSGLGVRGQGQGQRSASAVKVRVRVRGQG*GQGLRSEGRV...


In [62]:
def combine_columns_str(df, cols):
    return df[cols].apply(lambda row: ''.join(row.values.astype(str)), axis=1).str.cat()

comb_col = ['Frame1','Frame2','Frame3']
df5L = combine_columns_str(telo5L,comb_col)
df12R = combine_columns_str(telo12R,comb_col)
df17R = combine_columns_str(telo17R,comb_col)
df19L = combine_columns_str(telo19L,comb_col)

In [63]:
combined = df5L + df12R + df17R + df19L
print(len(combined))

205583


In [64]:
# amino acids
aa = ['A','B','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y','Z']

#all frames & telos
def aa_count(dataframe):
    for letter in aa:
        print(f'{letter}: {dataframe.count(letter)}')

aa_count(combined)

A: 780
B: 0
C: 273
D: 20
E: 3634
F: 1650
G: 70802
H: 13
I: 31
K: 526
L: 26511
M: 7
N: 1
P: 152
Q: 966
R: 32504
S: 2298
T: 60
V: 34390
W: 2655
Y: 64
Z: 0


In [65]:
# indiv telo print
#print('5L')
#aa_count(df5L)
#print('12R')
#aa_count(df12R)
#print('17R')
#aa_count(df17R)
#print('19L')
#aa_count(df19L)

19L
A: 304
B: 0
C: 53
D: 8
E: 502
F: 581
G: 17708
H: 5
I: 8
K: 164
L: 7017
M: 2
N: 0
P: 10
Q: 51
R: 7500
S: 469
T: 13
V: 8090
W: 861
Y: 11
Z: 0


In [66]:
comb_col = ['Frame1']
df5L = combine_columns_str(telo5L,comb_col)
df12R = combine_columns_str(telo12R,comb_col)
df17R = combine_columns_str(telo17R,comb_col)
df19L = combine_columns_str(telo19L,comb_col)

frame1_comb = df5L + df12R + df17R + df19L

#for letter in aa:
#    print(f'{letter}: {frame1_comb.count(letter)}')

A: 322
B: 0
C: 83
D: 9
E: 1358
F: 515
G: 23782
H: 1
I: 7
K: 210
L: 8509
M: 1
N: 1
P: 53
Q: 332
R: 11102
S: 618
T: 33
V: 11863
W: 917
Y: 17
Z: 0


In [67]:
# pd.concat "stacks" the sheets onto each other
# melt turns the 3 separate columns into one column with a variable
# 'id_vars' being the variables we want unchanged
bigdf=pd.concat([telo5L, telo12R, telo17R, telo19L]).melt(id_vars=['Name','Mat/Pat','Population','Sex','Revcom?','Telomere'], var_name='Frame',value_name='Sequence')
#print(bigdf)

                        Name Mat/Pat            Population     Sex Revcom?  \
0    HG.002.pat.cur.20211005     pat             Caucasian    male     yes   
1    HG.002.mat.cur.20211005     mat             Caucasian    male     yes   
2      NA19240.pri.mat.f1_v2     mat               Yoruban  female     yes   
3      HG02622.pri.mat.f1_v2     mat               Gambian  female      no   
4      NA18906.alt.pat.f1_v2     pat               Yoruban  female      no   
..                       ...     ...                   ...     ...     ...   
397    HG00673.pri.mat.f1_v2     mat     Han Chinese South    male     yes   
398    HG03453.alt.pat.f1_v2     pat  Mende (Sierra Leone)  female     yes   
399    HG02080.pri.mat.f1_v2     mat            Vietnamese  female      no   
400    NA20129.pri.mat.f1_v2     mat      African American  female     yes   
401    HG03492.pri.mat.f1_v2     mat    Punjabi (Pakistan)    male      no   

    Telomere   Frame                                           

In [79]:
# iterate by row
newdf = pd.DataFrame()

for i, row in bigdf.iterrows():
    # call sequence
    sequence = row['Sequence']
    # separate by *
    seqs = sequence.split("*")
    # check lengths
    for snippet in seqs:
        # if length > 50, print row & segment
        if len(snippet)>=30:
            # add snippet to sequence series
            row_copy = row.copy()
            row_copy['Snippet']=snippet
            # add row to new df
            newdf = pd.concat([newdf, row_copy[['Name','Population','Revcom?','Telomere','Frame','Snippet']].to_frame().T], ignore_index=True)
            #pd.concat([saveddf,row],axis=0)
            #print(row[['Name','Revcom?','Telomere','Snippet']])
            #print()
        else:
            continue

#print(newdf)

                         Name          Population Revcom? Telomere   Frame  \
0     HG.002.pat.cur.20211005           Caucasian     yes       5L  Frame1   
1     HG.002.pat.cur.20211005           Caucasian     yes       5L  Frame1   
2     HG.002.pat.cur.20211005           Caucasian     yes       5L  Frame1   
3     HG.002.pat.cur.20211005           Caucasian     yes       5L  Frame1   
4     HG.002.mat.cur.20211005           Caucasian     yes       5L  Frame1   
...                       ...                 ...     ...      ...     ...   
1436    HG03492.pri.mat.f1_v2  Punjabi (Pakistan)      no      19L  Frame3   
1437    HG03492.pri.mat.f1_v2  Punjabi (Pakistan)      no      19L  Frame3   
1438    HG03492.pri.mat.f1_v2  Punjabi (Pakistan)      no      19L  Frame3   
1439    HG03492.pri.mat.f1_v2  Punjabi (Pakistan)      no      19L  Frame3   
1440    HG03492.pri.mat.f1_v2  Punjabi (Pakistan)      no      19L  Frame3   

                                                Snippet  
0    

In [80]:
# Saves sequences to a csv
#newdf.to_csv('long_sequences.csv', index=False)

In [1]:
# Collect all snippets and count occurences of AA

# Create a set that holds AAs that show up <100 times
# Iterate thru snippet dataframe
# If snippet contains anything in AA, add snippet/row to new df